# Pytorch Dataset
Using Pytorch's `Dataset` and `DataLoader` we can build a data-serving pipeline to our model

## Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os,sys
import time
from IPython.core.debugger import set_trace

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
sys.dont_write_bytecode = True

In [ ]:
import joblib, copy
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from pprint import pprint

import pdb
from tqdm.autonotebook import tqdm

In [ ]:
import holoviews as hv
from holoviews import opts

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], shared_axes=False),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], shared_axes=False),
    opts.Image('mask', alpha=0.4),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
    opts.Curve( tools=['hover'], active_tools=['wheel_zoom'], padding=0.1),

)

In [ ]:
this_nb_path = Path(os.getcwd())
ROOT = this_nb_path.parent
SCRIPTS = ROOT/'src'
paths2add = [this_nb_path, SCRIPTS]

print("Project root: ", str(ROOT))
print("this nb path: ", str(this_nb_path))
print('Scripts folder: ', str(SCRIPTS))

for p in paths2add:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(str(p), "added to the path\n")
        
# print(sys.path)

In [ ]:
import ipywidgets
from ipywidgets import interact
def f(x):
    return x

interact(f, x=10)


---
## Import PyTorch


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

### [Optional] Set visible device

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES']='3'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# Random seed helper
from helpers import random_seed

## Globals

In [ ]:
IMGNET_MEAN = [0.485, 0.456, 0.406]
IMGNET_STD = [0.229, 0.224, 0.225]
TRAIN_DATA_DIR = Path('../data/cocolike/train/')
TEST_DATA_DIR = Path('../data/cocolike/test/')

## PyTorch Version

### 1. Dataset and DataLoader

In [ ]:
import datasets as seg_ds

- Define train and validation datasets without any transforms

In [ ]:
train_ds = seg_ds.SegDataset(
    ids_file=TRAIN_DATA_DIR/'train_ids.txt', 
    x_dir=TRAIN_DATA_DIR/'images',
    y_dir=TRAIN_DATA_DIR/'segmentations',
    verbose=True
)

- Visualization

### 2. Transforms

Compose transforms each of which will operate on numpy array (3dimensional with order h,w,nC)

In [ ]:
from model.seg_transforms import TailPadder
from PIL import Image

- Define x,y transformations

In [ ]:
RANDOM_ROT_DEGREE = 20
PAD_OUT_SIZE = 500
FILL = 255

tr_x_tsfm = transforms.Compose([
    transforms.ToPILImage(),
    TailPadder(PAD_OUT_SIZE, fill=FILL),
    
     # Data augs
#      transforms.RandomRotation(RANDOM_ROT_DEGREE, fill=FILL),
#      transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0, hue=0),
     transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0),
     transforms.ToTensor(),
     transforms.Normalize(mean=IMGNET_MEAN, std=IMGNET_STD),
])

tr_y_tsfm = transforms.Compose([
    lambda np_2d: transforms.ToPILImage()(np_2d[:,:,None]),
    TailPadder(PAD_OUT_SIZE, fill=FILL),
    
     # Data augs' geometric transforms only 
#      transforms.RandomRotation(RANDOM_ROT_DEGREE, fill=FILL),
     lambda pil_y: torch.from_numpy(np.asarray(pil_y))
])

val_x_tsfm = transforms.Compose([
    transforms.ToPILImage(),
    TailPadder(PAD_OUT_SIZE, fill=FILL),
     transforms.ToTensor(),
     transforms.Normalize(mean=IMGNET_MEAN, std=IMGNET_STD),
])

val_y_tsfm = transforms.Compose([
    lambda np_2d: transforms.ToPILImage()(np_2d[:,:,None]),
    TailPadder(PAD_OUT_SIZE, fill=FILL),
     lambda pil_y: torch.from_numpy(np.asarray(pil_y))
])

---

- Transformation + Dataset

In [ ]:
sample_train_ds = seg_ds.SegDataset(
        ids_file=TRAIN_DATA_DIR/'sample_train_ids.txt', 
        x_dir=TRAIN_DATA_DIR/'images',
        y_dir=TRAIN_DATA_DIR/'tf_segmentation',
        transforms = [tr_x_tsfm, tr_y_tsfm],
        verbose=True
)

sample_val_ds = seg_ds.SegDataset(
        ids_file=TRAIN_DATA_DIR/'sample_val_ids.txt', 
        x_dir=TRAIN_DATA_DIR/'images',
        y_dir=TRAIN_DATA_DIR/'tf_segmentation',
        transforms = [val_x_tsfm, val_y_tsfm],
        verbose=True
)

In [ ]:
train_ds = seg_ds.SegDataset(
        ids_file=TRAIN_DATA_DIR/'train_ids.txt', 
        x_dir=TRAIN_DATA_DIR/'images',
        y_dir=TRAIN_DATA_DIR/'segmentations',
        transforms = [tr_x_tsfm, tr_y_tsfm],
        verbose=True
    )

val_ds = seg_ds.SegDataset(
        ids_file=TRAIN_DATA_DIR/'val_ids.txt', 
        x_dir=TRAIN_DATA_DIR/'images',
        y_dir=TRAIN_DATA_DIR/'segmentations',
        transforms = [val_x_tsfm, val_y_tsfm],
        verbose=True
    )

test_ds = seg_ds.SegDataset(
    ids_file=TEST_DATA_DIR/'test_ids.txt',
    x_dir=TEST_DATA_DIR/'images',
    y_dir='',
    get_label=False,
    transforms = [val_x_tsfm, None],
    verbose=True
)

### 4. Transformed Dataset + Dataloader

In [ ]:
bs = 16
train_dl_params = {'batch_size': bs,
                   'shuffle': True,
#           'num_workers': 4
            }

In [ ]:
sample_train_dl = DataLoader(sample_train_ds, **train_dl_params)
sample_val_dl = DataLoader(sample_val_ds)
sample_dataloaders = {
    'train': sample_train_dl,
    'val': sample_val_dl,
}

In [ ]:
train_dl = DataLoader(train_ds, **train_dl_params)
val_dl = DataLoader(val_ds)
test_dl = DataLoader(test_ds)
dataloaders = {'train': train_dl,
               'val': val_dl,
               'test': test_dl}

### 5. Model Architecture

In [ ]:
from torchsummary import summary
import torchvision.models as tvmodels
from model.fcn import fcn32s, get_fcn16, get_pretrained_fcn16
from model.helpers import save_checkpt
from functools import partial

In [ ]:
from model.helpers import freeze_all, freeze_with_substr, unfreeze_all, get_frozen, get_trainable

In [ ]:
n_classes = 21
seed = 1
pretrain=True

In [ ]:
model = get_fcn16(device=DEVICE, seed=seed, pretrain=pretrain)
## freeze all layers 
## replace the last classifier
unfreeze_all(model.parameters())
freeze_with_substr(model, 'conv_block')

In [ ]:
summary(model, input_size=(3, 500,500))

In [ ]:
param_names = []
for p in model.named_parameters():
    param_names.append(p[0])
pprint(param_names)

In [ ]:
# loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255, reduction='mean')
# lr = 1e-5
# weight_decay = 0.01
# optimizer = optim.SGD(model.parameters(), lr=lr)
# optimizer = optim.Adam(model.parameters(), lr=lr)
# optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

### 7. Learning Rate Finder

- Exponentially increase learning rate at each iteration from a wide range (eg. 1e-8 to 1) to find a good max_lr for Cyclic Triangle Learning rate scheduling to be used for Train and Val loop 
- model: pretrained weights from VGG16
    - CONV_BLOCKs are frozen
- batch_size: 16
- loss_fn uses (approximatedly) inverse_freq class weights

In [ ]:
from scheduler.experiment import lr_range_test
from scheduler.utils import get_mult_factor

In [ ]:
# weighted loss function
loss_weights = torch.from_numpy(np.array([0.025]+[1.]*(n_classes-1))).float().to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(weight=loss_weights, ignore_index=255, reduction='mean')

In [ ]:
dl = train_dl
max_lr = 1.
min_lr = 1e-6
n_epochs = 1
n_iters = n_epochs * len(dl)
mult_factor = get_mult_factor(min_lr, max_lr, n_iters)
print(f'num of iters: {n_iters}, mult_factor: {mult_factor}')

In [ ]:
# model = get_fcn16(device=DEVICE, seed=seed, pretrain=pretrain)
optimizer = optim.SGD(model.parameters(),lr=min_lr)
lr_gen = optim.lr_scheduler.ExponentialLR(optimizer, gamma=mult_factor)

beta = 0.3 #relative weight on old_average_loss
lrs, losses, avg_losses = lr_range_test(model, dl, loss_fn, optimizer, lr_gen, DEVICE, n_iters=n_iters)

In [ ]:
hv.Curve(losses)

In [ ]:
hv.Curve(avg_losses)

In [ ]:
lrs[70]

## First round of training 

- max_lr is determined from lr_range_test
- using triangle, cyclic lr, train (+val) 

- model: pretrained fcn16
- loss_fn: cross entropy with class weights
- batch_size: 16


In [ ]:
from scheduler.cyclic import CyclicTLR
from run import run

In [ ]:
dl = train_dl

loss_weights = torch.from_numpy(np.array([0.025]+[1.]*(n_classes-1))).float().to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(weight=loss_weights, ignore_index=255, reduction='mean')

In [ ]:
max_lr = 0.021374546404928113 #lrs[70] #0.0124 0.007
divide_factor = 3.
min_lr = max_lr/divide_factor
stepsize_in_epoch = 1
stepsize =  stepsize_in_epoch * len(dl) # stepsize of lr-scheduler in iteration unit
print('stepsize: ', stepsize)

In [ ]:
n_cycles = 30
lr_test_epochs = (2*stepsize_in_epoch) * n_cycles
print('total train epoch: ', lr_test_epochs)

params = dict(max_epoch= lr_test_epochs,
             batch_size=dl.batch_size,
             fill_value=FILL)

# Need to get a clean model
model = get_fcn16(device=DEVICE, seed=seed, pretrain=pretrain)
## freeze all layers, train the last classifier #todo: experiement with a clean classifier
# unfreeze_all(model.parameters())
freeze_with_substr(model, 'conv_block')
# model = get_fcn16(device=DEVICE, seed=seed, pretrain=pretrain)
optimizer = optim.SGD(model.parameters(),lr=min_lr)
lr_scheduler = CyclicTLR(optimizer, min_lr, max_lr, stepsize)

exp_lrs = []
_, result = run(model, dataloaders, loss_fn, optimizer,
                lr_scheduler, DEVICE, params, exp_lrs)

In [ ]:
(
#     hv.Curve(lrs) *
    hv.Curve(result['train']['loss']) 
)

In [ ]:
train_losses = result['train']['loss']
train_accs = result['train']['acc']
val_losses = result['val']['loss']
val_accs = result['val']['acc']

In [ ]:
hv.Curve(result['train']['loss'])  * hv.Curve(result['val']['loss']).opts(color='red')

In [ ]:
(
    hv.Curve(result['train']['acc'])  
    *
    hv.Curve(result['val']['acc']).opts(color='red')
)

In [ ]:
#debugging
from evaluate import evaluate

In [ ]:
res = evaluate(model, val_dl, loss_fn, DEVICE, params)

## Weighted loss function version


In [ ]:
dl = train_dl
loss_weights = torch.from_numpy(np.array([0.025]+[1.]*(n_classes-1))).float().to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(weight=loss_weights, ignore_index=255, reduction='mean')

In [ ]:
max_lr = 0.008
divide_factor = 3.
min_lr = max_lr/divide_factor
stepsize_in_epoch = 1
stepsize =  stepsize_in_epoch * len(dl) # stepsize of lr-scheduler in iteration unit
print('stepsize: ', stepsize)

n_cycles = 10
lr_test_epochs = (2*stepsize_in_epoch) * n_cycles
print('total train epoch: ', lr_test_epochs)

params = dict(max_epoch= lr_test_epochs,
             batch_size=dl.batch_size,
             fill_value=FILL)

model = get_fcn16(DEVICE, seed=1)
optimizer = optim.SGD(model.parameters(),lr=min_lr)
lr_scheduler = CyclicTLR(optimizer, min_lr, max_lr, stepsize)

lrs = []
_, result = run(model, dataloaders, loss_fn, optimizer,
                lr_scheduler, DEVICE, params, lrs)

In [ ]:
(
#     hv.Curve(lrs) *
    hv.Curve(result['train']['loss']) 
)

In [ ]:
train_losses = result['train']['loss']
train_accs = result['train']['acc']
val_losses = result['val']['loss']
val_accs = result['val']['acc']

In [ ]:
hv.Curve(result['train']['loss'])  * hv.Curve(result['val']['loss']).opts(color='red')

In [ ]:
(
    hv.Curve(result['train']['acc'])  
#     *
#     hv.Curve(result['val']['acc']).opts(color='red')
)